In [1]:
import math
from civilpy.general import units
from civilpy.structural.steel import W, MC
from civilpy.structural.arema import LoadRatingMember
from termcolor import colored
from pprint import pprint

# Workbook Layout

8 Sheets,

1. Global
1. Superstructure DC
1. Girder
1. Floor Beam
1. Deck Plate
1. Abut End Floor Beam
1. Floor Beam Prop
1. Floor Beam - Non Bracing - Void

# Inputs

## Global Definitions

Contains the various Design Inputs, and summaries of the various checks being
performed. # //TODO - Assumes precast ties?

In [ ]:
global_values = GlobalDefinitions()

# Prints the global definitions used in calculations
pprint(global_values.__dict__)

In [ ]:
loads = LoadDefinitions()

loads.jack_pt_offset=4*units.ft

# Prints the load definitions used in calculations
pprint(loads.__dict__)

In [ ]:
tpg = TPG(load_values=loads)

# Prints the remaining definitions not used above
pprint(tpg.__dict__)

# Calcs

## Global

## Preliminary Girder Calcs

This was moved ahead of DC Calcs because later calcs depend on the results here

## Superstructure DC Calcs

## Remaining Girder Calcs/Checks

In [ ]:
V_tot

bearing_stress_act = (V_tot / bearing_stiff_area).to('psi')

# Longitudinal Stiffeners
if bearing_stress_act <= bearing_stiff_F_a:
    print(colored(f'OK, Stress Ratio: {bearing_stress_act/bearing_stiff_F_a}', 'green'))
else:
    print(colored('No Good - End Bearing Compression Check', 'red'))

In [ ]:
# Bearing Stiff Weld      AREMA 15-1.7.7.a
fillet_weld_area = 4 * 0.707 * bearing_stiffener_fillet_weld_leg * (girder_web_height - 2 * bearing_stiffener_corner_clip)
V_tot

weld_stress_act = (V_tot / fillet_weld_area).to('psi')

allowable_weld_stress = min(19000 * units('psi'), 0.35*F_y) # AREMA Table 15-1-14

# Bearing Stiff Weld Check
if weld_stress_act <= allowable_weld_stress:
    print(colored(f'OK, Stress Ratio: {weld_stress_act/allowable_weld_stress}', 'green'))
else:
    print(colored('No Good - Bearing Stiff Weld Check', 'red'))

In [ ]:
# Bearing Stiff bearing area
stiff_bearing_area = 2 * bearing_stiffener_thickness_tsb * (bearing_stiffener_width_bsb - 2 * bearing_stiffener_corner_clip)

In [ ]:
stiff_bearing_area
V_tot
bearing_area_stress_act = (V_tot / stiff_bearing_area).to('psi')
bearing_area_stress_all = (.83 * F_y).to('psi')

# Bearing area allowable stress
if bearing_area_stress_act <= bearing_area_stress_all:
    print(colored(f'OK, Stress Ratio: {bearing_area_stress_act/bearing_area_stress_all}', 'green'))
else:
    print(colored('No Good - Bearing Stiff Weld Check', 'red'))

## Floor Beam Calcs

In [ ]:
# Dead Load
# Floor Beams
floorbeam.weight

# Diaphragms
diaphragm.weight = 61 * units('lbf/ft') # //TODO - Doesn't match section: W16x89
diaphragm_quantity = 1 # //TODO - Why is this only 1 in excel?

diaphragm_length = floorbeam_spacing

lateral_bracing_length = 8.20 * units('ft') # //TODO - why is this redefined and lower

diaphragm_P = diaphragm.weight*diaphragm_quantity*diaphragm_length
bracing_P = lateral_bracing.weight * bracing_quant * lateral_bracing_length

floorbeam_dl_V = (floorbeam.weight * girder_spacing) / 2
floorbeam_dl_M = floorbeam.weight * girder_spacing ** 2 / 8

diaphragm_dl_V = diaphragm_P / 2
diaphragm_dl_M = diaphragm_P * girder_spacing / 4

lateral_bracing_V = bracing_P / 2
lateral_bracing_M = bracing_P * girder_spacing / 4

In [ ]:
# Floor assembly between stop plates
floor_load_on_fbs
ballast_plates_clear_space

floor_assembly_V = ballast_plates_clear_space * floor_load_on_fbs * floorbeam_spacing / 2
floor_assembly_M = (floor_load_on_fbs * floorbeam_spacing * ballast_plates_clear_space) / 4 * (girder_spacing - ballast_plates_clear_space / 2) 

In [ ]:
# Floor assembly over stop plates
floor_load_over_stop_plates
fb_applied_width = (girder_spacing - ballast_plates_clear_space) / 2

floor_over_stop_pl_V = floor_load_over_stop_plates * fb_applied_width * floorbeam_spacing
floor_over_stop_pl_M = floor_load_over_stop_plates * fb_applied_width ** 2 / 2 * floorbeam_spacing

In [ ]:
# Track assembly weight
track_area_load_on_fbs
ballast_plates_clear_space

track_area_load_V = track_area_load_on_fbs*ballast_plates_clear_space/2*floorbeam_spacing
track_area_load_M = track_area_load_on_fbs * ballast_plates_clear_space / 4 * (girder_spacing - ballast_plates_clear_space / 2) * floorbeam_spacing

In [ ]:
total_fb_dl_shear = sum([
    floorbeam_dl_V,
    diaphragm_dl_V,
    lateral_bracing_V,
    floor_assembly_V,
    floor_over_stop_pl_V,
    track_area_load_V
])

In [ ]:
total_fb_dl_moment = sum([
    floorbeam_dl_M,
    diaphragm_dl_M,
    lateral_bracing_M,
    floor_assembly_M,
    floor_over_stop_pl_M,
    track_area_load_M
])

In [ ]:
# Live Load
# AREMA 15-1.3.4.2.3 Alternative live load axel
fb_P_ll = 1.15*fb_A*floorbeam_spacing/fb_S
fb_P_2_ll = fb_P_ll / 2

fb_V_ll = fb_P_2_ll
fb_M_ll = fb_P_ll * fb_a / 2

In [ ]:
# Impact Live Load
fb_impact_ll_percent = (40 - 3 * girder_spacing.magnitude ** 2 / 1600)/100
fb_P_2_ll

fb_P_imp_ll = fb_impact_ll_percent * fb_P_2_ll * ballasted_deck_reduction

fb_M_imp = fb_P_imp_ll * fb_a

In [ ]:
# Rocking Load  AREMA 1.3.5.d
rocking_percent
fb_P_2_ll
R1_rocking = fb_P_2_ll * rocking_percent * (1 - 2 * a / girder_spacing)

fb_M_re = R1_rocking * a

In [ ]:
# Wind Force on Loaded Bridge
wind_load = 300 * units('lbf/ft')
fb_pws = wind_load*h*floorbeam_spacing/railroad_gage

fb_wind_R1 = fb_pws * (1-2*a/girder_spacing)

M_wl = fb_wind_R1 * fb_a

In [ ]:
# Required Section Properties  # AREMA Table 15-1-11
fb_M_tot_1 = sum([total_fb_dl_moment, fb_M_ll, fb_M_imp, M_re])
fb_M_tot_2 = sum([total_fb_dl_moment, fb_M_ll, fb_M_imp, M_re, M_wl])/1.25
fb_M_tot = max(fb_M_tot_1, fb_M_tot_2)

fb_S_x_req = (fb_M_tot / (0.55 * F_y)).to('in^3')

In [ ]:
# Holes - Assume 4 x 1" holes in web for diaphragm
D1 = 3 * units('in') # //TODO - is this a constant?
D2 = 3 * units('in') + D1

I_holes_web = (diaphragm_num_holes/2*(diaphragm_dia_hole*floorbeam.web_thickness)*(D1)**2)+(diaphragm_num_holes/2*(diaphragm_dia_hole*floorbeam.web_thickness)*(D2)**2)

In [ ]:
I_holes_flange = lateral_num_holes * lateral_dia_holes * floorbeam.flange_thickness * (floorbeam.depth / 2 - floorbeam.flange_thickness / 2) ** 2

In [ ]:
I_net = floorbeam.I_x - I_holes_web - I_holes_flange

In [ ]:
I_net

S_x_net = I_net / (floorbeam.depth / 2)

In [ ]:
# Allowable Stress (AREMA Table 15-1-12)
Fb_ten = 0.55 * F_y

# Compression in extreme fiber
Fb_comp_1 = 0.55*F_y-(0.55*F_y**2/(6.3*math.pi**2*E_steel))*(diaphragm_spacing_max/floorbeam.r_y)**2
Fb_comp_2 = (0.131 * math.pi * E_steel / (diaphragm_spacing_max * floorbeam.depth * (1+poisson_ratio) ** .5 / (floorbeam.flange_thickness * floorbeam.flange_width))).to('psi')
Fb_comp_3 = 0.55 * F_y

F_b_comp = (min(Fb_comp_3, max(Fb_comp_1, Fb_comp_2))).to('ksi')

f_b_act = (fb_M_tot / S_x_net).to('ksi')

# Bearing area allowable stress
if f_b_act < F_b_comp:
    print(colored(f'OK, Stress Ratio: {f_b_act/F_b_comp}', 'green'))
else:
    print(colored('No Good - Floor Beam Allowable Stress Check', 'red'))

In [ ]:
# End Shear
R_dl = total_fb_dl_shear
R_ll = fb_P_ll
R_imp = fb_P_imp_ll * 2
R_re = R1_rocking
R_wl = fb_wind_R1

R_tot = sum([R_dl, R_ll, R_imp, R_re, R_wl])

# Assume 6x1" dia holes in web to connect to girder
fs_net = (R_tot/((floorbeam.depth-web_conn_girder_holes_num*web_conn_girder_holes_dia)*floorbeam.web_thickness)).to('ksi')

F_v = (0.35 * F_y).to('ksi')

# Bearing area allowable stress
if fs_net < F_v:
    print(colored(f'OK, Stress Ratio: {fs_net/F_v}', 'green'))
else:
    print(colored('No Good - Floor Beam End Shear Check', 'red'))

In [ ]:
# Bolted Connection - Checked in sep. calc, they account for combined efect so no 1.25 increase
# AREMA 15 1.5.9.a.1 # //TODO - Investigate this standard
R_tot

In [ ]:
# Floor beam fatigue
fb_M_ll
fb_fat_imp_M = assumed_mean_impact_perc * fb_M_imp
fb_M_fat = fb_M_ll + fb_fat_imp_M

fb_live_load_stress_range = (fb_M_fat / S_x_net).to('ksi')

# N >= 2,000,000 Category B Table 15-1-10 Detail 2.2
fb_fsr = 16 * units('ksi')

# Bearing area allowable stress
if fb_live_load_stress_range < fb_fsr:
    print(colored(f'OK, Stress Ratio: {fb_live_load_stress_range/fb_fsr}', 'green'))
else:
    print(colored('No Good - Floor Beam Fatigue Check', 'red'))

In [ ]:
# Deflection
girder_spacing
floorbeam.I_x
E_steel

fb_defl_M = fb_M_ll + fb_M_imp + fb_M_re

fb_w = 8 * fb_defl_M / (girder_spacing ** 2)

fb_deflection = ((5 * fb_w * girder_spacing ** 4) / (384 * E_steel * floorbeam.I_x)).to('in')
max_deflection = (girder_spacing / 640).to('in')

# Deflection Check
if fb_deflection <= max_deflection:
    print(colored(f'OK, Stress Ratio: {fb_deflection/max_deflection}', 'green'))
else:
    print(colored('No Good - Floor Beam Fatigue Check', 'red'))

## Deck Plate Calcs

In [ ]:
# Key Dimensions
track_distribution_width = tie_design_width+min_ballast_below_tie # AREMA 1.3.4.2.2.b
track_dist_length = 3 * units('ft') + min_ballast_below_tie # AREMA 1.3.4.2.2.a

In [ ]:
# Dead Loads
flooring_load = floor_load_on_fbs
track_load = track_assembly_load_on_deck_plate

In [ ]:
# Live Load
axel_load
longitudinal_dist = track_dist_length
lateral_dist = track_distribution_width

dist_axel_load = axel_load / (longitudinal_dist*lateral_dist)

In [ ]:
# Impact Load # AREMA 15.1.3.5
L = floorbeam_spacing
deck_pl_impact_L = (40 - (3*L**2 / (1600 * units('ft^2')))) * .9

impact_load = (dist_axel_load * deck_pl_impact_L / 100).to('lbf/ft^2')
impact_load

In [ ]:
# Rocking Effect
rocking_percent

rocking_effect = rocking_percent * dist_axel_load / 2

In [ ]:
deck_pl_dead_load = flooring_load + track_load
deck_pl_live_load_impact_re = dist_axel_load + impact_load + rocking_effect

total_dist_load = deck_pl_dead_load + deck_pl_live_load_impact_re

In [ ]:
# Compute Forces and Stresses over a 1' unit width
flange_width = min(end_floorbeam.flange_width, floorbeam.flange_width)

floorbeam_spacing
flange_width.to('ft')

M1_w = (1/12*((3*floorbeam_spacing*flange_width/2)-(floorbeam_spacing**2)-(3*flange_width**2/8))).to('ft^2')
M2_w = floorbeam_spacing ** 2 / 24

gov_M_w = max(abs(M1_w), abs(M2_w))

deck_pl_dl_M = gov_M_w * deck_pl_dead_load
deck_pl_ll_M = gov_M_w * deck_pl_live_load_impact_re
deck_pl_tot_M = deck_pl_dl_M + deck_pl_ll_M

deck_pl_S_x = deck_plate_width * deck_plate_thickness ** 2 / 6

# //TODO - Check Units on this one
deck_pl_F_b = (deck_pl_tot_M * (12*units('in')) / deck_pl_S_x).to('psi')

deck_pl_F_b_allow = (0.55 * F_y)

# Stress Check
if deck_pl_F_b <= deck_pl_F_b_allow:
    print(colored(f'OK, Stress Ratio: {deck_pl_F_b/deck_pl_F_b_allow}', 'green'))
else:
    print(colored('No Good - Floor Plate Allowable Stress Check', 'red'))

In [ ]:
# Floor Plate Deflection
deck_plate_width
deck_plate_thickness
deck_plate_I_x = deck_plate_width * deck_plate_thickness ** 3 / 12
deck_plate_span_L = floorbeam_spacing
deck_pl_live_load_impact_re

deck_pl_live_load_deflection = (deck_pl_live_load_impact_re*units('in')*floorbeam_spacing**4)/(384*E_steel*deck_plate_I_x)
deck_pl_allow_deflection = (floorbeam_spacing / 640).to('in').to('in')

deck_pl_live_load_deflection = deck_pl_live_load_deflection.to('in')

# Deflection Check
if deck_pl_live_load_deflection <= deck_pl_allow_deflection:
    print(colored(f'OK, Stress Ratio: {deck_pl_live_load_deflection/deck_pl_allow_deflection}', 'green'))
else:
    print(colored('No Good - Floor Plate Deflection Check', 'red'))

## End Floor Beam Calcs

In [ ]:
# Floor Beams
end_floorbeam.weight
jack_pt_offset
# end_bearing_stiff_width = (end_floorbeam.flange_width - end_floorbeam.web_thickness) / 2 * 4 / 4
end_bearing_stiff_width = 5.75 * units('in')
bearing_stiffener_t
bearing_stiffener_corner_clip
bearing_stiff_weld_leg

# Geometrics
end_fb_spacing = 2.6 * units('ft') # //TODO - References VOID fb - non-bracing sheet
flooring_on_girder = end_fb_spacing/2+(floor_length-span_length)/2
girder_spacing
ballast_plates_clear_space
diaphragm_spacing_max
railroad_gage
F_y
F_u
E_steel
unit_weight_steel
railroad_gage # //TODO - Defined twice in excel

In [ ]:
# Dead Load
# Diaphragms
diaphragm.weight = 61 * units('lbf/ft') # //TODO - Doesn't match section: W16x89
diaphragm_quantity = 1 # //TODO - Why is this only 1 in excel?

diaphragm_length = floorbeam_spacing / 2

lateral_bracing_length = 8.20 * units('ft') # //TODO - why is this redefined and lower


diaphragm_P = diaphragm.weight*diaphragm_quantity*diaphragm_length


end_floorbeam_dl_V = (end_floorbeam.weight * girder_spacing) / 2
end_floorbeam_dl_M = end_floorbeam.weight * girder_spacing ** 2 / 8

end_diaphragm_dl_V = diaphragm_P / 2
end_diaphragm_dl_M = diaphragm_P * end_bearing_stiff_width / 4 # //TODO - Verify End stiff width

In [ ]:
# //TODO - Verify no bracing on end floorbeams
# bracing_P = lateral_bracing.weight * bracing_quant * lateral_bracing_length
# end_lateral_bracing_V = bracing_P / 2
# end_lateral_bracing_M = bracing_P * girder_spacing / 4

In [ ]:
# Floor assembly between stop plates
floor_load_on_fbs
ballast_plates_clear_space

end_floor_assembly_V = ballast_plates_clear_space * floor_load_on_fbs * flooring_on_girder / 2 # //TODO - Verify Flooring on girder, not fb spacing
end_floor_assembly_M = (floor_load_on_fbs * flooring_on_girder * ballast_plates_clear_space) / 4 * (girder_spacing - ballast_plates_clear_space / 2) 

In [ ]:
# Floor assembly over stop plates
floor_load_over_stop_plates
end_fb_applied_width = (girder_spacing - ballast_plates_clear_space) / 2

end_floor_over_stop_pl_V = floor_load_over_stop_plates * end_fb_applied_width * flooring_on_girder
end_floor_over_stop_pl_M = floor_load_over_stop_plates * end_fb_applied_width ** 2 / 2 * flooring_on_girder

In [ ]:
# Track assembly weight
track_area_load_on_fbs
ballast_plates_clear_space

end_track_area_load_V = track_area_load_on_fbs*ballast_plates_clear_space/2*flooring_on_girder
end_track_area_load_M = track_area_load_on_fbs * ballast_plates_clear_space / 4 * (girder_spacing - ballast_plates_clear_space / 2) * flooring_on_girder

In [ ]:
total_end_fb_dl_shear = sum([
    end_floorbeam_dl_V,
    end_diaphragm_dl_V,
    end_floor_assembly_V,
    end_floor_over_stop_pl_V,
    end_track_area_load_V
])

In [ ]:
total_end_fb_dl_moment = sum([
    end_floorbeam_dl_M,
    end_diaphragm_dl_M,
    end_floor_assembly_M,
    end_floor_over_stop_pl_M,
    end_track_area_load_M
])

In [ ]:
# Live Load
# AREMA 15-1.3.4.2.3 Alternative live load axel
end_fb_P_ll = 1.15*fb_A*floorbeam_spacing/fb_S
end_fb_P_2_ll = fb_P_ll / 2

end_fb_A = fb_A
end_fb_D = floorbeam_spacing
end_fb_S = fb_S
end_fb_a = fb_a

end_fb_M_ll = end_fb_P_ll * end_fb_a / 2

In [ ]:
# Impact Live Load
end_fb_impact_ll_percent = (40 - 3 * girder_spacing.magnitude ** 2 / 1600)/100
end_fb_P_2_ll

end_fb_P_imp_ll = end_fb_impact_ll_percent * end_fb_P_2_ll * ballasted_deck_reduction

end_fb_M_imp = fb_P_imp_ll * fb_a

In [ ]:
# Rocking Load  AREMA 1.3.5.d
rocking_percent
end_fb_P_2_ll
end_R1_rocking = end_fb_P_2_ll * rocking_percent * (1 - 2 * a / girder_spacing)

end_fb_M_re = R1_rocking * a

In [ ]:
# Wind Force on Loaded Bridge
wind_load = 300 * units('lbf/ft')
end_fb_pws = wind_load*h*floorbeam_spacing/railroad_gage

end_fb_wind_R1 = end_fb_pws * (1-2*a/girder_spacing)

end_fb_M_wl = end_fb_wind_R1 * fb_a

In [ ]:
# Required Section Properties and gov moment  # AREMA Table 15-1-11
end_fb_M_tot_1 = sum([total_end_fb_dl_moment, end_fb_M_ll, end_fb_M_imp, end_fb_M_re])
end_fb_M_tot_2 = sum([total_end_fb_dl_moment, end_fb_M_ll, end_fb_M_imp, end_fb_M_re, end_fb_M_wl])/1.25
end_fb_M_tot = max(end_fb_M_tot_1, fb_M_tot_2)

In [ ]:
# Governing shear # //TODO - Ensure this isn't need in other calc
end_fb_V_tot_1 = sum([total_end_fb_dl_shear, end_fb_P_2_ll, end_fb_P_imp_ll, end_R1_rocking])
end_fb_V_tot_2 = sum([total_end_fb_dl_shear, end_fb_P_2_ll, end_fb_P_imp_ll, end_R1_rocking, end_fb_wind_R1])/1.25
end_fb_V_tot = max(end_fb_V_tot_1, end_fb_V_tot_2)

In [ ]:
total_dead_load
dead_load_per_jack = total_dead_load / 4
jack_pt_offset

In [ ]:
# Shear and moment doubled to meet requirements of 11-1.8.1
jacking_V_tot = dead_load_per_jack * 2
jacking_M_tot = 2 * dead_load_per_jack * jack_pt_offset

In [ ]:
# Required Section Properties

end_fb_gov_M = max(jacking_M_tot, end_fb_M_tot)
end_fb_gov_V = max(jacking_V_tot, end_fb_V_tot)

end_fb_S_x_req = (end_fb_gov_M / (0.55 * F_y)).to('in^3')

In [ ]:
# Holes - Assume 4 x 1" holes in web for diaphragm
D1 = 3 * units('in') # //TODO - is this a constant?
D2 = 3 * units('in') + D1

I_holes_web = (diaphragm_num_holes/2*(diaphragm_dia_hole*end_floorbeam.web_thickness)*(D1)**2)+(diaphragm_num_holes/2*(diaphragm_dia_hole*end_floorbeam.web_thickness)*(D2)**2)

In [ ]:
I_holes_web

In [ ]:
I_holes_flange = lateral_num_holes * lateral_dia_holes * floorbeam.flange_thickness * (floorbeam.depth / 2 - floorbeam.flange_thickness / 2) ** 2

In [ ]:
I_net = floorbeam.I_x - I_holes_web - I_holes_flange

In [ ]:
I_net

S_x_net = I_net / (floorbeam.depth / 2)

In [ ]:
# Allowable Stress (AREMA Table 15-1-12)
Fb_ten = 0.55 * F_y

# Compression in extreme fiber
Fb_comp_1 = 0.55*F_y-(0.55*F_y**2/(6.3*math.pi**2*E_steel))*(diaphragm_spacing_max/floorbeam.r_y)**2
Fb_comp_2 = (0.131 * math.pi * E_steel / (diaphragm_spacing_max * floorbeam.depth * (1+poisson_ratio) ** .5 / (floorbeam.flange_thickness * floorbeam.flange_width))).to('psi')
Fb_comp_3 = 0.55 * F_y

F_b_comp = (min(Fb_comp_3, max(Fb_comp_1, Fb_comp_2))).to('ksi')

f_b_act = (fb_M_tot / S_x_net).to('ksi')

# Bearing area allowable stress
if f_b_act < F_b_comp:
    print(colored(f'OK, Stress Ratio: {f_b_act/F_b_comp}', 'green'))
else:
    print(colored('No Good - Floor Beam Allowable Stress Check', 'red'))

In [ ]:
# End Shear
R_dl = total_fb_dl_shear
R_ll = fb_P_ll
R_imp = fb_P_imp_ll * 2
R_re = R1_rocking
R_wl = fb_wind_R1

R_tot = sum([R_dl, R_ll, R_imp, R_re, R_wl])

# Assume 6x1" dia holes in web to connect to girder
fs_net = (R_tot/((floorbeam.depth-web_conn_girder_holes_num*web_conn_girder_holes_dia)*floorbeam.web_thickness)).to('ksi')

F_v = (0.35 * F_y).to('ksi')

# Bearing area allowable stress
if fs_net < F_v:
    print(colored(f'OK, Stress Ratio: {fs_net/F_v}', 'green'))
else:
    print(colored('No Good - Floor Beam End Shear Check', 'red'))

In [ ]:
# Bolted Connection - Checked in sep. calc, they account for combined efect so no 1.25 increase
# AREMA 15 1.5.9.a.1 # //TODO - Investigate this standard
R_tot

In [ ]:
# Floor beam fatigue
fb_M_ll
fb_fat_imp_M = assumed_mean_impact_perc * fb_M_imp
fb_M_fat = fb_M_ll + fb_fat_imp_M

fb_live_load_stress_range = (fb_M_fat / S_x_net).to('ksi')

# N >= 2,000,000 Category B Table 15-1-10 Detail 2.2
fb_fsr = 16 * units('ksi')

# Bearing area allowable stress
if fb_live_load_stress_range < fb_fsr:
    print(colored(f'OK, Stress Ratio: {fb_live_load_stress_range/fb_fsr}', 'green'))
else:
    print(colored('No Good - Floor Beam Fatigue Check', 'red'))

In [ ]:
# Deflection
girder_spacing
floorbeam.I_x
E_steel

fb_defl_M = fb_M_ll + fb_M_imp + fb_M_re

fb_w = 8 * fb_defl_M / (girder_spacing ** 2)

fb_deflection = ((5 * fb_w * girder_spacing ** 4) / (384 * E_steel * floorbeam.I_x)).to('in')
max_deflection = (girder_spacing / 640).to('in')

# Deflection Check
if fb_deflection <= max_deflection:
    print(colored(f'OK, Stress Ratio: {fb_deflection/max_deflection}', 'green'))
else:
    print(colored('No Good - Floor Beam Fatigue Check', 'red'))